In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Retreive test set, iaa_dataset and annotations dataset

In [ ]:
testset = pd.read_csv("../../Data/resultsLRDutch_withId.csv", index_col=0)
testset = testset.reset_index(drop=True)

In [ ]:
iaa_dataset = pd.read_excel("../../Data/tweets.xlsx")
iaa_dataset = tweets[tweets['GR'] != 'unclear']

In [ ]:
annotations = pd.read_pickle("../../Data/annotations.pkl")
annotations = annotations.astype({'Tweet_id': 'int64'})

Select only the tweets that are in the test set

In [ ]:
annotations = annotations[annotations["Tweet_id"].isin(testset["Tweet_id"])]

In [ ]:
iaa_dataset = iaa_dataset[iaa_dataset["Tweet_id"].isin(testset["Tweet_id"])]

Create dataset annotated by one person

In [ ]:
one_annotator_dataset = annotations.drop_duplicates(subset=["Tweet_id"])

In [ ]:
one_annotator_dataset

,Sentiment,text_clean,Tweet_id,Annotator_id
0,neutral,ik heb 13u aan een stuk gewerkt come at me,1,67
5,positive,"#Leestip van het weekend, wat mij betreft. @YL...",6,67
7,neutral,kijkt naar ATP World Tour Final,8,67
8,positive,Mooie prestatie philippeblockma ! Ontdek onze ...,9,67
12,positive,Koesterdoos! Ontdek dit geweldige project op @...,13,67
...,...,...,...,...
13182,negative,als ik 3 keer uw volgverzoek weiger dan ga ik ...,2583,26
13184,neutral,Herlees: Sweden's Rise to Heaven (38): De Zwee...,2585,26
13185,neutral,Alwéér hommeles met prins Laurent: is het balo...,2586,26
13192,negative,Ma jong gij zijt een belachelijk ventje,2593,26


Create Dataset where easy tweets are annotated by 1 person and hard annotated by 5

In [ ]:
golden_rule_pred = testset.merge(iaa_dataset, on=["Tweet_id"])
golden_rule_pred = golden_rule_pred[["GR","Pred", "Tweet_id"]]

In [ ]:
golden_rule_pred

,GR,Pred,Tweet_id
0,positive,True,562
1,neutral,False,2832
2,neutral,True,184
3,neutral,True,1434
4,negative,True,1759
...,...,...,...
579,negative,False,1312
580,negative,False,2993
581,neutral,True,200
582,negative,True,2794


In [ ]:
one_anno_with_GR = one_annotator_dataset.merge(golden_rule_pred[golden_rule_pred["Pred"]], how="left")

In [ ]:
one_anno_with_GR["Sentiment"] = one_anno_with_GR["GR"].fillna(one_anno_with_GR["Sentiment"])

In [ ]:
thesis_dataset = one_anno_with_GR[["Sentiment", "Tweet_id"]]

In [ ]:
oad = one_annotator_dataset.merge(golden_rule_pred, on="Tweet_id")

In [ ]:
one_anno_with_GR

,Sentiment,text_clean,Tweet_id,Annotator_id,GR,Pred
0,neutral,ik heb 13u aan een stuk gewerkt come at me,1,67,NaN,NaN
1,positive,"#Leestip van het weekend, wat mij betreft. @YL...",6,67,positive,True
2,neutral,kijkt naar ATP World Tour Final,8,67,NaN,NaN
3,positive,Mooie prestatie philippeblockma ! Ontdek onze ...,9,67,positive,True
4,positive,Koesterdoos! Ontdek dit geweldige project op @...,13,67,NaN,NaN
...,...,...,...,...,...,...
579,negative,als ik 3 keer uw volgverzoek weiger dan ga ik ...,2583,26,negative,True
580,neutral,Herlees: Sweden's Rise to Heaven (38): De Zwee...,2585,26,neutral,True
581,neutral,Alwéér hommeles met prins Laurent: is het balo...,2586,26,neutral,True
582,negative,Ma jong gij zijt een belachelijk ventje,2593,26,negative,True


In [ ]:
number_of_true_pred = one_anno_with_GR["Pred"].sum()

In [ ]:
random_ids = one_anno_with_GR["Tweet_id"].sample(number_of_true_pred)

In [ ]:
one_anno_with_GR["Tweet_id"].isin(random_ids)

0      False
1       True
2       True
3       True
4      False
       ...  
579     True
580     True
581     True
582    False
583     True
Name: Tweet_id, Length: 584, dtype: bool

In [ ]:
temp = one_annotator_dataset.merge(golden_rule_pred[golden_rule_pred["Tweet_id"].isin(random_ids)], how="left")

In [ ]:
temp["Sentiment"] = temp["GR"].fillna(temp["Sentiment"])

In [ ]:
random_performance = temp[["Sentiment", "Tweet_id"]]

Cost and accuracy calculations

In [ ]:
one_dataset_num_correct = oad[oad["Sentiment"] == oad["GR"]]["Tweet_id"].count()

In [ ]:
one_dataset_cost = len(one_annotator_dataset)

In [ ]:
rp = random_performance.merge(golden_rule_pred, on="Tweet_id")

In [ ]:
random_num_correct = rp[rp["Sentiment"] == rp["GR"]]["Tweet_id"].count()


In [ ]:
td = thesis_dataset.merge(golden_rule_pred, on="Tweet_id")

In [ ]:
thesis_dataset_num_correct = td[td["Sentiment"] == td["GR"]]["Tweet_id"].count()


In [ ]:
thesis_dataset_num_correct

543

In [ ]:
random_num_correct

538

In [ ]:
thesis_dataset_cost = 80 + 239*5

In [ ]:
gr_dataset_cost = len(golden_rule_pred) * 5

In [ ]:
one_dataset_acc = one_dataset_num_correct / 584

In [ ]:
thesis_dataset_acc = thesis_dataset_num_correct / 584

In [ ]:
rand_data_acc = random_num_correct / 584

In [ ]:
from sklearn.preprocessing import scale

In [ ]:
name = ["One Annotator", "Random Model with Task Routing", "Task Routing", "Golden Rule"]
accuracy = [round(one_dataset_acc,2),round(rand_data_acc,2), round(thesis_dataset_acc,2),1]
cost = [one_dataset_cost/gr_dataset_cost, thesis_dataset_cost/gr_dataset_cost,thesis_dataset_cost/gr_dataset_cost ,gr_dataset_cost/gr_dataset_cost]
cost_anno = [one_dataset_cost, thesis_dataset_cost ,thesis_dataset_cost,gr_dataset_cost]

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(name='Accuracy',
           x=name,
           y=accuracy,
           text=accuracy,
           textposition='auto'))

fig.add_trace(
    go.Bar(name='Cost',
           x=name,
           y=cost,
           text=cost_anno,
           textposition='auto'))

fig.update_layout(
    template="plotly_white",
    title_text='Cost and Accuracy Comparison',
    title_font_size=30,
    legend = dict(font = dict(size = 18)),
    width=1200, height=600,
    font=dict(
        family="Helvetica",
    )
)
fig.update_xaxes(title_font=dict(size=22))
fig.update_yaxes(title_font=dict(size=22))
fig.show()

In [ ]:
name = ["One Annotator", "Task Routing (1 or 5 annotators)", "Golden Rule (5 annotators)"]
accuracy = [round(one_dataset_acc,2), round(thesis_dataset_acc,2),1]
cost = [one_dataset_cost/gr_dataset_cost,thesis_dataset_cost/gr_dataset_cost ,gr_dataset_cost/gr_dataset_cost]
cost_anno = [one_dataset_cost, thesis_dataset_cost,thesis_dataset_cost,gr_dataset_cost]

rand_acc = round(rand_data_acc,2)
rand_cost = thesis_dataset_cost/gr_dataset_cost
rand_name = "Random Model with Task Routing (1 or 5 annotators)"

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
           x=cost,
           y=accuracy,
           text=name,
           mode="lines+markers+text",
           textposition="top center",
           showlegend=False
           ))
fig.add_trace(
    go.Scatter(
           x=[rand_cost],
           y=[rand_acc],
           mode='markers+text',
           text=rand_name,
           textposition="bottom center",
           showlegend=False
           ))

fig.update_layout(
    template="plotly_white",
    title_text='Cost and Accuracy Comparison Dutch',
    title_font_size=30,
    legend = dict(font = dict(size = 18)),
    width=1200, height=600,
    font=dict(
        family="Helvetica",
    ),
    xaxis_title="Cost",
    yaxis_title="Accuracy",
)
fig.update_traces(marker=dict(size=12))